In [2]:
!pip install trimesh

# importing sys
import sys
# adding src to the system path
sys.path.insert(0, './src')

import os
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from DataSetsLoader import DataSetsLoader
from PointNetFactory import PointNetFactory

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.9/660.9 KB 28.2 MB/s eta 0:00:00
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Global variables
NUM_POINTS = 2048
NUM_CLASSES = 10
NUM_POINT_AXIS = 3
BATCH_SIZE = 32

# Use hardware accelerator for training
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("GPUs Available: ", physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Load Datasets
datasets_loader = DataSetsLoader()
train_points, test_points, train_labels, test_labels, CLASS_MAP = datasets_loader.transform_to_tensorflow_dataset()

In [ ]:
# Shuffle and Augment Datasets
def augment(points, label):
        # jitter points
        points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
        # shuffle points
        points = tf.random.shuffle(points)
        return points, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

In [ ]:
point_net_factory = PointNetFactory()

In [ ]:
# Build the PointNet model
model = point_net_factory.create_model(NUM_POINTS, NUM_POINT_AXIS, NUM_CLASSES)
# model.summary()

In [ ]:
# Train Model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_dataset, epochs=20, validation_data=test_dataset)

In [ ]:
# Save Model
model.save_weights("./model.h5")

# Load Model
new_model = point_net_factory.create_model(NUM_POINTS, NUM_POINT_AXIS, NUM_CLASSES)
new_model.load_weights("./model.h5")

In [ ]:
# Predict Results
data = test_dataset.take(1)

points, labels = list(data)[0]
points = points[:10, ...]
labels = labels[:10, ...]

# run test data through model
preds = new_model.predict(points)
preds = tf.math.argmax(preds, -1)

points = points.numpy()

# plot points with predicted class and label
fig = plt.figure(figsize=(15, 10))
for i in range(10):
    ax = fig.add_subplot(2, 5, i + 1, projection="3d")
    ax.scatter(points[i, :, 0], points[i, :, 1], points[i, :, 2])
    ax.set_title(
        "pred: {:}, label: {:}".format(
            CLASS_MAP[preds[i].numpy()], CLASS_MAP[labels.numpy()[i]]
        )
    )
    ax.set_axis_off()
plt.show()